# Spark SQL

In [1]:
spark.sparkContext.setLogLevel("ERROR")

### Check all the `databases` present

In [2]:
spark.sql('SHOW databases').show()           # This is coming from AWS Glue Catalog

23/07/10 17:10:00 INFO HiveConf: Found configuration file file:/etc/spark/conf.dist/hive-site.xml
23/07/10 17:10:00 WARN HiveConf: HiveConf of name hive.server2.thrift.url does not exist
23/07/10 17:10:01 INFO AWSGlueClientFactory: Using region from ec2 metadata : us-east-2


+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
+--------------------+



### Check the present `database` selected

In [3]:
spark.catalog.currentDatabase() # This with show the presently selected `database`

'default'

### Check all the `tables` present

In [4]:
spark.sql('SHOW tables').show() # In the `default` Database

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



### Select a particular `database`

In [5]:
spark.sql('USE dev_feedback')   # Select a differet database

DataFrame[]

### Show the `tables` within the `database`

In [6]:
spark.sql('SHOW tables').show()  # Show all the tables within the database 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



### Create a database

In [7]:
spark.sql('CREATE DATABASE IF NOT EXISTS my_db_spark')

23/07/10 17:10:03 INFO FileUtils: Creating directory if it doesn't exist: hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db


DataFrame[]

In [8]:
spark.sql('SHOW databases').show()

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
|         my_db_spark|
+--------------------+



In [9]:
spark.sql('USE my_db_spark') 

DataFrame[]

In [10]:
spark.sql('SHOW tables').show()           # Show all the tables within the database 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [11]:
spark.sql('USE db_youtube_raw')
spark.sql('SHOW tables').show(truncate=False)           

+--------------+-----------------------------+-----------+
|namespace     |tableName                    |isTemporary|
+--------------+-----------------------------+-----------+
|db_youtube_raw|raw_statistics               |false      |
|db_youtube_raw|raw_statistics_reference_data|false      |
+--------------+-----------------------------+-----------+



In [12]:
spark.sql('SHOW databases').show()        # Other tables are coming from AWS Glue Catalog

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
|         my_db_spark|
+--------------------+



In [13]:
# Check if a particular database is present or not (in case the list is long)

( 
    spark
    .sql('SHOW databases')
    .filter("namespace = 'my_db_spark'")
    .show() 
)

+-----------+
|  namespace|
+-----------+
|my_db_spark|
+-----------+



In [14]:
# Check if a list of databases are present or not (in case the list is long)

( 
    spark
    .sql('SHOW databases')
    .filter("namespace LIKE 'db%'")
    .show() 
)

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
+--------------------+



### Create a DataFrame

In [15]:
spark.sql('USE my_db_spark') 
spark.catalog.currentDatabase()    # Check the present database (which is selected)

'my_db_spark'

In [16]:
spark.sql('SHOW tables').show(truncate=False)           

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [17]:
spark.sql('CREATE TABLE my_db_spark.orders \
               (order_id integer, \
                order_date string, \
                customer_id integer, \
                order_status string)')

23/07/10 17:10:09 INFO SQLStdHiveAccessController: Created SQLStdHiveAccessController for session context : HiveAuthzSessionContext [sessionString=80b63cd3-0979-414a-b58a-483422bcc5a3, clientType=HIVECLI]
23/07/10 17:10:09 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/07/10 17:10:09 INFO AWSCatalogMetastoreClient: Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl to org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook
23/07/10 17:10:09 INFO AWSGlueClientFactory: Using region from ec2 metadata : us-east-2
23/07/10 17:10:10 INFO AWSGlueClientFactory: Using region from ec2 metadata : us-east-2
23/07/10 17:10:10 INFO FileUtils: Creating directory if it doesn't exist: hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders


DataFrame[]

In [18]:
spark.catalog.currentDatabase()    # Check the present database (which is selected)

'my_db_spark'

In [19]:
spark.sql('SHOW tables').show()

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+



> Now you can go to a new shell and run this 
```python
spark.sql('USE my_db_spark')
spark.sql('SHOW tables').show()
+-----------+---------+-----------+                                             
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+
```

### Create a `TempView` using the DataFrame

In [20]:
# Lets first create a DF using that we will create a TempView
data_set = 's3://fcc-spark-example/dataset/2023/orders.csv'
df = spark.read.csv('s3://fcc-spark-example/dataset/2023/orders.csv', header=True, inferSchema=True)

# Creating a TempView
df.createOrReplaceTempView('my_db_spark.orders_view')

In [21]:
spark.sql('SHOW tables').show()

+-----------+-----------+-----------+
|  namespace|  tableName|isTemporary|
+-----------+-----------+-----------+
|my_db_spark|     orders|      false|
|           |orders_view|       true|
+-----------+-----------+-----------+



> Now you can go to a new shell and run this **(WE WON'T SEE THE TEMP VIEW)**
```python
spark.sql('USE my_db_spark')
spark.sql('SHOW tables').show()
+-----------+---------+-----------+                                             
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_db_spark|   orders|      false|
+-----------+---------+-----------+
```

### Insert data from the `TempView` into the new `Table` (the persistent table)

In [22]:
spark.sql("INSERT INTO orders \
            SELECT * \
            FROM orders_view")

23/07/10 17:10:19 INFO log: Updating table stats fast for orders                
23/07/10 17:10:19 INFO log: Updated size of table orders to 2862089


DataFrame[]

In [23]:
spark.sql("SELECT * FROM orders").show(5)

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
+--------+-------------------+-----------+---------------+
only showing top 5 rows



### Describe `Table`

In [24]:
spark.sql("DESCRIBE TABLE orders").show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|    order_id|      int|   null|
|  order_date|   string|   null|
| customer_id|      int|   null|
|order_status|   string|   null|
+------------+---------+-------+



In [25]:
# For the persistent table 
spark.sql("DESCRIBE EXTENDED orders").show()              # Its a managed table

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|            order_id|                 int|   null|
|          order_date|              string|   null|
|         customer_id|                 int|   null|
|        order_status|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|         my_db_spark|       |
|               Table|              orders|       |
|               Owner|              hadoop|       |
|        Created Time|Mon Jul 10 17:10:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|  Spark 3.3.0-amzn-1|       |
|                Type|             MANAGED|       |
|            Provider|                hive|       |
|          Statistics|       2862089 bytes|       |
|            Location|hdfs://ip-172-31-...|       |
|       Serd

In [26]:
# For the TempView
spark.sql("DESCRIBE EXTENDED orders_view").show()              

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|         order_id|      int|   null|
|       order_date|timestamp|   null|
|order_customer_id|      int|   null|
|     order_status|   string|   null|
+-----------------+---------+-------+



### Check the underline data in `HDFS` (Managed Table)

In [27]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse

Found 1 items
drwxrwxrwt   - hadoop spark          0 2023-07-10 17:10 hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db


In [28]:
%%bash

hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders

Found 1 items
-rwxrwxrwt   1 hadoop spark    2862089 2023-07-10 17:10 hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders/part-00000-a4c7eb3f-7c59-447c-8186-4c73de69baef-c000


In [30]:
%%bash 

hadoop fs -head hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders/part-00000-a4c7eb3f-7c59-447c-8186-4c73de69baef-c000

12013-07-25 00:00:0011599CLOSED
22013-07-25 00:00:00256PENDING_PAYMENT
32013-07-25 00:00:0012111COMPLETE
42013-07-25 00:00:008827CLOSED
52013-07-25 00:00:0011318COMPLETE
62013-07-25 00:00:007130COMPLETE
72013-07-25 00:00:004530COMPLETE
82013-07-25 00:00:002911PROCESSING
92013-07-25 00:00:005657PENDING_PAYMENT
102013-07-25 00:00:005648PENDING_PAYMENT
112013-07-25 00:00:00918PAYMENT_REVIEW
122013-07-25 00:00:001837CLOSED
132013-07-25 00:00:009149PENDING_PAYMENT
142013-07-25 00:00:009842PROCESSING
152013-07-25 00:00:002568COMPLETE
162013-07-25 00:00:007276PENDING_PAYMENT
172013-07-25 00:00:002667COMPLETE
182013-07-25 00:00:001205CLOSED
192013-07-25 00:00:009488PENDING_PAYMENT
202013-07-25 00:00:009198PROCESSING
212013-07-25 00:00:002711PENDING
222013-07-25 00:00:00333COMPLETE
232013-07-25 00:00:004367PENDING_PAYMENT
242013-07-25 00:00:0011441CLOSED
252013-07-25 00:00:009503CLOSED
262013-07-25 00:00:007562COMPLET

### Deleting the `table`

In [31]:
spark.sql("DROP TABLE orders")

23/07/10 17:11:46 INFO GlueMetastoreClientDelegate: Initiating drop table partitions


DataFrame[]

In [32]:
# spark.sql("DESCRIBE TABLE orders").show() # It will throw an ERROR 

In [33]:
## It should give an error (Here we used MANAGED Table, and hence the data and metadata both got deleted when we ran DROP)

# %%bash

# hadoop fs -ls hdfs://ip-172-31-2-35.us-east-2.compute.internal:8020/user/spark/warehouse/my_db_spark.db/orders

# Clean Up 

In [34]:
spark.sql("DROP DATABASE my_db_spark")

DataFrame[]

In [35]:
spark.catalog.currentDatabase()

'my_db_spark'

In [36]:
spark.sql('USE default')

DataFrame[]

In [37]:
spark.catalog.currentDatabase()

'default'

In [38]:
spark.sql("SHOW databases").show()

+--------------------+
|           namespace|
+--------------------+
|db_youtube_analytics|
| db_youtube_cleansed|
|      db_youtube_raw|
|             default|
|        dev_feedback|
+--------------------+

